In [ ]:
def cross_validation_with_degree(y, tx, k_fold, hyperparameters, method="ridge"):
    """
    Perform cross-validation to tune hyperparameters for different regression methods, including degree.

    Args:
        y: numpy array of shape=(N,)
        tx: numpy array of shape=(N, D)
        k_fold: Number of cross-validation folds
        hyperparameters: Dictionary containing hyperparameters specific to the selected method,
            including "degree" for polynomial regression methods.
        method: string, the name of the regression method ("ridge", "least_squares", "mean_squared_error_gd", "mean_squared_error_sgd", "logistic", or "reg_logistic").

    Returns:
        best_hyperparameters: Dictionary with the best hyperparameters
        best_loss: The lowest loss obtained during cross-validation
    """
    num_samples = len(y)
    fold_size = num_samples // k_fold
    best_loss = float("inf")
    best_hyperparameters = {}

    for hyperparameter_set in hyperparameters:
        total_loss = 0.0

        for fold in range(k_fold):
            val_start = fold * fold_size
            val_end = (fold + 1) * fold_size
            val_indices = np.arange(val_start, val_end)
            train_indices = np.delete(np.arange(num_samples), val_indices)

            y_train, tx_train = y[train_indices], tx[train_indices]
            y_val, tx_val = y[val_indices], tx[val_indices]

            if "degree" in hyperparameter_set:
                degree = hyperparameter_set["degree"]
                tx_train = build_poly(tx_train, degree)
                tx_val = build_poly(tx_val, degree)

            if method == "ridge":
                lambda_ = hyperparameter_set["lambda"]
                w = ridge_regression(y_train, tx_train, lambda_)
            elif method == "least_squares":
                w, _ = least_squares(y_train, tx_train)
            elif method == "mean_squared_error_gd":
                initial_w = hyperparameter_set["initial_w"]
                max_iters = hyperparameter_set["max_iters"]
                gamma = hyperparameter_set["gamma"]
                w, _ = mean_squared_error_gd(y_train, tx_train, initial_w, max_iters, gamma)
            elif method == "mean_squared_error_sgd":
                initial_w = hyperparameter_set["initial_w"]
                max_iters = hyperparameter_set["max_iters"]
                gamma = hyperparameter_set["gamma"]
                batch_size = hyperparameter_set["batch_size"]
                w, _ = mean_squared_error_sgd(y_train, tx_train, initial_w, max_iters, gamma, batch_size)
            elif method == "logistic":
                initial_w = hyperparameter_set["initial_w"]
                max_iters = hyperparameter_set["max_iters"]
                gamma = hyperparameter_set["gamma"]
                w, _ = logistic_regression(y_train, tx_train, initial_w, max_iters, gamma)
            elif method == "reg_logistic":
                lambda_ = hyperparameter_set["lambda"]
                initial_w = hyperparameter_set["initial_w"]
                max_iters = hyperparameter_set["max_iters"]
                gamma = hyperparameter_set["gamma"]
                w, _ = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iters, gamma)
            else:
                raise ValueError("Invalid regression method.")

            if method in ["ridge", "logistic", "reg_logistic"]:
                val_loss = compute_loss(y_val, tx_val, w, "log")
            else:
                val_loss = np.sqrt(2 * compute_mse(y_val, tx_val, w))

            total_loss += val_loss

        average_loss = total_loss / k_fold

        if average_loss < best_loss:
            best_loss = average_loss
            best_hyperparameters = hyperparameter_set

    return best_hyperparameters, best_loss


In [ ]:
# Define a list of hyperparameters for different regression methods
hyperparameters = []

# Ridge Regression Hyperparameters
ridge_hyperparams = [
    {"method": "ridge", "lambda": 0.1, "degree": 1},
    {"method": "ridge", "lambda": 0.01, "degree": 2},
    # Add more hyperparameters for ridge regression as needed
]
hyperparameters.extend(ridge_hyperparams)

# Least Squares Hyperparameters
least_squares_hyperparams = [
    {"method": "least_squares"},
    # Add more hyperparameters for least squares as needed
]
hyperparameters.extend(least_squares_hyperparams)

# Mean Squared Error (Gradient Descent) Hyperparameters
mse_gd_hyperparams = [
    {"method": "mean_squared_error_gd", "initial_w": initial_w_value, "max_iters": max_iters_value, "gamma": gamma_value, "degree": 1},
    # Add more hyperparameters for MSE with GD as needed
]
hyperparameters.extend(mse_gd_hyperparams)

# Mean Squared Error (Stochastic Gradient Descent) Hyperparameters
mse_sgd_hyperparams = [
    {"method": "mean_squared_error_sgd", "initial_w": initial_w_value, "max_iters": max_iters_value, "gamma": gamma_value, "batch_size": batch_size_value, "degree": 1},
    # Add more hyperparameters for MSE with SGD as needed
]
hyperparameters.extend(mse_sgd_hyperparams)

# Logistic Regression Hyperparameters
logistic_hyperparams = [
    {"method": "logistic", "initial_w": initial_w_value, "max_iters": max_iters_value, "gamma": gamma_value, "degree": 1},
    # Add more hyperparameters for logistic regression as needed
]
hyperparameters.extend(logistic_hyperparams)

# Regularized Logistic Regression Hyperparameters
reg_logistic_hyperparams = [
    {"method": "reg_logistic", "lambda": 0.1, "initial_w": initial_w_value, "max_iters": max_iters_value, "gamma": gamma_value, "degree": 1},
    # Add more hyperparameters for regularized logistic regression as needed
]
hyperparameters.extend(reg_logistic_hyperparams)

# You can continue to add more hyperparameters as needed for each method.

# Now, the `hyperparameters` list contains sets of hyperparameters for all six methods.


In [ ]:
# Define the list of hyperparameters for each regression method
ridge_hyperparams = [
    {"method": "ridge", "lambda": 0.1, "degree": 1},
    {"method": "ridge", "lambda": 0.01, "degree": 2},
    # Add more hyperparameters for ridge regression as needed
]

least_squares_hyperparams = [
    {"method": "least_squares"},
    # Add more hyperparameters for least squares as needed
]

mse_gd_hyperparams = [
    {"method": "mean_squared_error_gd", "initial_w": initial_w_value, "max_iters": max_iters_value, "gamma": gamma_value, "degree": 1},
    # Add more hyperparameters for MSE with GD as needed
]

mse_sgd_hyperparams = [
    {"method": "mean_squared_error_sgd", "initial_w": initial_w_value, "max_iters": max_iters_value, "gamma": gamma_value, "batch_size": batch_size_value, "degree": 1},
    # Add more hyperparameters for MSE with SGD as needed
]

logistic_hyperparams = [
    {"method": "logistic", "initial_w": initial_w_value, "max_iters": max_iters_value, "gamma": gamma_value, "degree": 1},
    # Add more hyperparameters for logistic regression as needed
]

reg_logistic_hyperparams = [
    {"method": "reg_logistic", "lambda": 0.1, "initial_w": initial_w_value, "max_iters": max_iters_value, "gamma": gamma_value, "degree": 1},
    # Add more hyperparameters for regularized logistic regression as needed
]

# Call cross_validation_with_degree for each regression method
best_hyperparams_ridge, best_loss_ridge = cross_validation_with_degree(y, tx, k_fold, ridge_hyperparams, method="ridge")
best_hyperparams_least_squares, best_loss_least_squares = cross_validation_with_degree(y, tx, k_fold, least_squares_hyperparams, method="least_squares")
best_hyperparams_mse_gd, best_loss_mse_gd = cross_validation_with_degree(y, tx, k_fold, mse_gd_hyperparams, method="mean_squared_error_gd")
best_hyperparams_mse_sgd, best_loss_mse_sgd = cross_validation_with_degree(y, tx, k_fold, mse_sgd_hyperparams, method="mean_squared_error_sgd")
best_hyperparams_logistic, best_loss_logistic = cross_validation_with_degree(y, tx, k_fold, logistic_hyperparams, method="logistic")
best_hyperparams_reg_logistic, best_loss_reg_logistic = cross_validation_with_degree(y, tx, k_fold, reg_logistic_hyperparams, method="reg_logistic")

# You can access the best hyperparameters and losses for each method as needed.
